In [ ]:
from pathlib import Path
from typing import Any

# ASCII tree components
SPACE = "    "
BRANCH = "│   "
TEE = "├── "
LAST = "└── "


def build_tree_structure(paths: list[str] | dict[str, float]) -> dict[str, Any]:
    """Build a nested dictionary representing the directory tree structure."""
    tree = {}

    # Handle both list of strings and dict of string->float
    if isinstance(paths, dict):
        path_items = paths.items()
    else:
        path_items = [(path, None) for path in paths]

    for path_str, size in path_items:
        path = Path(path_str)
        parts = path.parts
        current = tree

        # Navigate through each part of the path
        for i, part in enumerate(parts):
            if part not in current:
                current[part] = {"_children": {}, "_size": None, "_is_file": False}

            # If this is the last part (the actual file/directory name)
            if i == len(parts) - 1:
                current[part]["_size"] = size
                current[part]["_is_file"] = True

            current = current[part]["_children"]

    return tree


def get_display_items(items: list[tuple], max_files: int, sort_by_size: bool = False) -> tuple:
    """
    Get the items to display based on limits and sorting preferences.

    Returns:
        (items_to_show, truncated_count)
    """
    if len(items) <= max_files:
        return items, 0

    if sort_by_size:
        # Sort by size (largest first), handling None sizes
        items_sorted = sorted(items, key=lambda x: x[1].get("_size", 0) or 0, reverse=True)
        return items_sorted[:max_files], len(items) - max_files
    else:
        # Just take the first items
        return items[:max_files], len(items) - max_files


def format_file_name(name: str, node_info: dict[str, Any], show_sizes: bool = False) -> str:
    """Format the file/directory name with optional size information."""
    if show_sizes and node_info.get("_size") is not None:
        size = node_info["_size"]
        if size >= 1024 * 1024 * 1024:  # GB
            size_str = f"{size / (1024**3):.1f}GB"
        elif size >= 1024 * 1024:  # MB
            size_str = f"{size / (1024**2):.1f}MB"
        elif size >= 1024:  # KB
            size_str = f"{size / 1024:.1f}KB"
        else:
            size_str = f"{size}B"
        return f"{name} ({size_str})"
    return name


def render_tree(
    tree_dict: dict[str, Any],
    prefix: str = "",
    max_depth: int | None = None,
    current_depth: int = 0,
    max_files_per_dir: int | None = None,
    sort_by_size: bool = False,
    show_sizes: bool = False,
) -> list[str]:
    """Recursively render the tree structure as ASCII art with limits."""
    lines = []

    # Check depth limit
    if max_depth is not None and current_depth >= max_depth:
        lines.append(prefix + TEE + "... (depth limit reached)")
        return lines

    items = list(tree_dict.items())

    # Apply per-directory file limit if specified
    if max_files_per_dir is not None and len(items) > max_files_per_dir:
        items_to_show, truncated_count = get_display_items(
            items, max_files_per_dir, sort_by_size
        )
    else:
        items_to_show = items
        truncated_count = 0

    for i, (name, node_info) in enumerate(items_to_show):
        # Determine if this is the last item at this level (considering truncation)
        is_last = i == len(items_to_show) - 1 and truncated_count == 0

        # Choose the appropriate pointer
        pointer = LAST if is_last else TEE

        # Format the name with size if requested
        display_name = format_file_name(name, node_info, show_sizes)

        # Add the current item
        lines.append(prefix + pointer + display_name)

        # If there are subitems, recurse
        if node_info["_children"]:
            # Choose the appropriate extension for the prefix
            extension = SPACE if is_last else BRANCH
            lines.extend(
                render_tree(
                    node_info["_children"],
                    prefix + extension,
                    max_depth,
                    current_depth + 1,
                    max_files_per_dir,
                    sort_by_size,
                    show_sizes,
                )
            )

    # Add truncation notice if files were truncated
    if truncated_count > 0:
        pointer = LAST
        lines.append(prefix + pointer + f"... ({truncated_count} more items)")

    return lines


def paths_to_tree(
    paths: list[str] | dict[str, float],
    max_depth: int | None = None,
    max_files_per_dir: int | None = None,
    sort_by_size: bool = False,
    show_sizes: bool = False,
) -> str:
    """
    Convert a list of file paths or dict of paths->sizes to an ASCII tree representation.

    Args:
        paths: List of file path strings OR dict of path -> file size in bytes
        max_depth: Maximum depth to display (None for unlimited)
        max_files_per_dir: Maximum files per directory to display (None for unlimited)
        sort_by_size: If True and max_files_per_dir is set, show largest files first
        show_sizes: If True, display file sizes next to filenames (requires dict input)

    Returns:
        String containing the ASCII tree representation

    Example:
        >>> paths = [
        ...     "src/main.py",
        ...     "src/utils/helpers.py",
        ...     "src/utils/config.py",
        ...     "tests/test_main.py",
        ...     "README.md"
        ... ]
        >>> print(paths_to_tree(paths, max_depth=2, max_files_per_dir=3))

        >>> path_sizes = {
        ...     "src/main.py": 1024,
        ...     "src/utils/helpers.py": 2048,
        ...     "README.md": 512
        ... }
        >>> print(paths_to_tree(path_sizes, sort_by_size=True, show_sizes=True))
    """
    if not paths:
        return ""

    # Build the tree structure
    tree_structure = build_tree_structure(paths)

    # Render the tree as ASCII art
    tree_lines = render_tree(
        tree_structure,
        max_depth=max_depth,
        max_files_per_dir=max_files_per_dir,
        sort_by_size=sort_by_size,
        show_sizes=show_sizes,
    )

    return "\n".join(tree_lines)


# Example usage
# Example 1: Basic usage with strings
example_paths = [
    "src/main.py",
    "src/utils/helpers.py",
    "src/utils/config.py",
    "src/utils/database.py",
    "src/utils/auth.py",
    "src/models/user.py",
    "src/models/database.py",
    "tests/test_main.py",
    "tests/test_utils.py",
    "tests/integration/test_api.py",
    "tests/integration/test_db.py",
    "docs/README.md",
    "docs/api/endpoints.md",
    "docs/api/authentication.md",
    "requirements.txt",
    ".gitignore",
]

print("1. Basic tree (no limits):")
print(paths_to_tree(example_paths))
print("\n" + "=" * 60 + "\n")

print("2. With depth limit (max_depth=2):")
print(paths_to_tree(example_paths, max_depth=2))
print("\n" + "=" * 60 + "\n")

print("3. With per-directory file limit (max_files_per_dir=3):")
print(paths_to_tree(example_paths, max_files_per_dir=3))
print("\n" + "=" * 60 + "\n")

# Example 2: Using dictionary with file sizes
path_sizes = {
    "src/main.py": 2048,
    "src/utils/helpers.py": 1024,
    "src/utils/config.py": 512,
    "src/utils/database.py": 4096,
    "src/utils/auth.py": 3072,
    "src/models/user.py": 1536,
    "tests/test_main.py": 2560,
    "README.md": 1024,
    "requirements.txt": 256,
}

print("4. With file sizes (sizes hidden by default):")
print(paths_to_tree(path_sizes))
print("\n" + "=" * 60 + "\n")

print("5. With file sizes displayed:")
print(paths_to_tree(path_sizes, show_sizes=True))
print("\n" + "=" * 60 + "\n")

print("6. Sorted by size, 2 files per directory, sizes shown:")
print(paths_to_tree(path_sizes, max_files_per_dir=2, sort_by_size=True, show_sizes=True))
print("\n" + "=" * 60 + "\n")

print("7. Sorted by size but sizes hidden (useful for organizing without displaying sizes):")
print(paths_to_tree(path_sizes, max_files_per_dir=2, sort_by_size=True, show_sizes=False))
print("\n" + "=" * 60 + "\n")

print("8. All limits combined:")
print(
    paths_to_tree(
        path_sizes, max_depth=3, max_files_per_dir=5, sort_by_size=True, show_sizes=True
    )
)

1. Basic tree (no limits):
├── src
│   ├── main.py
│   ├── utils
│   │   ├── helpers.py
│   │   ├── config.py
│   │   ├── database.py
│   │   └── auth.py
│   └── models
│       ├── user.py
│       └── database.py
├── tests
│   ├── test_main.py
│   ├── test_utils.py
│   └── integration
│       ├── test_api.py
│       └── test_db.py
├── docs
│   ├── README.md
│   └── api
│       ├── endpoints.md
│       └── authentication.md
├── requirements.txt
└── .gitignore


2. With depth limit (max_depth=2):
├── src
│   ├── main.py
│   ├── utils
│   │   ├── ... (depth limit reached)
│   └── models
│       ├── ... (depth limit reached)
├── tests
│   ├── test_main.py
│   ├── test_utils.py
│   └── integration
│       ├── ... (depth limit reached)
├── docs
│   ├── README.md
│   └── api
│       ├── ... (depth limit reached)
├── requirements.txt
└── .gitignore


3. With per-directory file limit (max_files_per_dir=3):
├── src
│   ├── main.py
│   ├── utils
│   │   ├── helpers.py
│   │   ├── config.py
│   │ 